#**Caso: Predicción de abandono de clientes en una empresa de telecomunicaciones**
**Contexto:**

Una empresa de telecomunicaciones quiere reducir la pérdida de clientes (churn). Para ello, te han proporcionado un conjunto de datos históricos sobre sus clientes actuales y pasados. Tu tarea es analizar estos datos y construir un modelo que prediga la probabilidad de que un cliente abandone la empresa.

🧭 Tareas a realizar

1. Análisis Exploratorio de Datos (EDA)

  * Describir el conjunto de datos: tipos de variables, valores nulos, estadísticas básicas.

  * Visualizaciones clave (distribuciones, correlaciones, etc.).

  * Identificar patrones relacionados con el abandono (Churn).

2. Limpieza y Preparación de Datos

  * Tratamiento de valores nulos.

  * Codificación de variables categóricas.

  * Normalización o estandarización si es necesario.

3. Modelado Predictivo

  * Separar conjunto de entrenamiento y prueba.

  * Probar al menos dos modelos de clasificación (ej: Regresión Logística, Árboles de Decisión, Random Forest, etc.).

  * Evaluar los modelos (precisión, recall, F1-score, curva ROC).

5. Interpretación y Recomendaciones

  * ¿Qué variables son más importantes para predecir el churn?

  * ¿Qué acciones propondrías a la empresa para reducir la pérdida de clientes?

6. Entrega

  * Un notebook Jupyter o Google Colab con el análisis completo.

  * Un resumen ejecutivo (máximo 1 página) explicando los hallazgos clave y recomendaciones, escrito para alguien no técnico.

In [26]:
# ------------------------- Librerias requeridas --------------------------------------------

# Tratamiento de datos
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score

# Visualización de datos
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import plotly.express as px

# Selección de variables y medición del desempeño
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import RandomizedSearchCV 

# Modelos Candidatos 
from sklearn.linear_model import LogisticRegression      # Regresión logística
from sklearn.ensemble import RandomForestClassifier      # Clasificador bosques aleatorios
from xgboost import XGBClassifier                        # XGBoost 

# Datos
#!pip install "kagglehub[pandas-datasets]"
import kagglehub as kg
from kagglehub import KaggleDatasetAdapter


#  Exploración inicial de los datos 

In [20]:
# Importación de los datos

file_path = "WA_Fn-UseC_-Telco-Customer-Churn.csv"

df = kg.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "blastchar/telco-customer-churn",
  file_path,
)

df.head(5)


C:\Users\gvand\AppData\Local\Temp\ipykernel_7672\393107368.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kg.load_dataset(


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
# Información general de los datos 
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [22]:
# Verificación de Nulos y duplicados
df.isna().sum()          
df.duplicated().sum()     

0

In [23]:
# Descripción estadística
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [ ]:
# Transformacion de la Variable "TotalCharges"
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan) # Este código fue necesario debido a que presentaba datos NaN
df['TotalCharges'] = df['TotalCharges'].astype(float) 

In [34]:
# Datos faltantes
df.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [35]:
# Eliminar datos "NaN"
data = df.dropna(subset=['TotalCharges'])

Conclusiones: 
- La base de datos no presenta valores nulos ni registros duplicados.
- El conjunto de datos está compuesto por 21 variables, las cuales se distribuyen entre variables numéricas, categóricas y binarias.
- El análisis estadístico muestra que solo el 16 % de los clientes son adultos mayores, con una antigüedad promedio de 32 meses y gastos mensuales medios de 64,76.
- Fue necesario realizar una primera transformación en la variable "TotalCharges", ya que aunque representa valores numéricos, originalmente estaba clasificada como texto (object). Esta conversión fue esencial para el análisis exploratorio.
- Dado que los valores NaN en la variable "TotalCharges" representan solo una fracción muy pequeña del total (11 de 7,043 registros), se optó por eliminar estas filas.
- Tras la eliminación de los 11 registros que presentaban valores "NaN" en la variable "TotalCharges", el conjunto de datos quedó conformado por 7,038 registros

# Análisis Exploratorio de Datos (EDA)

## Análisis Variable Objetivo

In [37]:
# Variable Respuesta (target)
print(df['Churn'].value_counts())
_ = df['Churn'].value_counts()

# Crear el gráfico de torta
fig = px.pie(values = _, names = _.index)
fig.show()

Churn
No     5174
Yes    1869
Name: count, dtype: int64


Conclusiones: 
- El 26.5% han renunciado al servicio, equivalente a un total de 1,869 clientes. 
- La distribución de clases binarias presenta un desequilibrio moderado. 

## Matriz de correlación